# Exercise 2 - Vaccindata

In [1]:
# importing all the relevant packages to complete my project
import pandas as pd # Pandas to be able to create and manipulate dataframes
import numpy as np # used for any mathematical functions that may be needed
import seaborn as sns # used for effective plotting with pandas
import plotly_express as px # used for a more interactive plotting
import matplotlib.pyplot as plt # used in conjunction with seaborn

In [2]:
# inporting excel sheet and creating a dataframe from it
df_covid_vaccine = pd.read_excel("../Lab_Covid/Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx","Vaccinerade kommun och ålder")
df_covid_vaccine.head()

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN


In [3]:
# Will now do some exploitative analysis on the dataset to see what it contains.
df_covid_vaccine.info()
df_covid_vaccine.columns,  df_covid_vaccine.index
# I can see that there is 2900 entries with 14 columns. Some of the columns have NaN values. There is also a mix of string, int and float values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Län                  2900 non-null   int64  
 1   Län_namn             2900 non-null   object 
 2   Kommun               2900 non-null   int64  
 3   Kommun_namn          2900 non-null   object 
 4   Ålder                2900 non-null   object 
 5   Befolkning           2900 non-null   int64  
 6   Antal minst 1 dos    2900 non-null   int64  
 7   Antal minst 2 doser  2900 non-null   int64  
 8   Antal 3 doser        2320 non-null   float64
 9   Antal 4 doser        870 non-null    float64
 10  Andel minst 1 dos    2900 non-null   float64
 11  Andel minst 2 doser  2900 non-null   float64
 12  Andel 3 doser        2320 non-null   float64
 13  Andel 4 doser        870 non-null    float64
dtypes: float64(6), int64(5), object(3)
memory usage: 317.3+ KB


(Index(['Län', 'Län_namn', 'Kommun', 'Kommun_namn', 'Ålder', 'Befolkning',
        'Antal minst 1 dos', 'Antal minst 2 doser', 'Antal 3 doser',
        'Antal 4 doser', 'Andel minst 1 dos', 'Andel minst 2 doser',
        'Andel 3 doser', 'Andel 4 doser'],
       dtype='object'),
 RangeIndex(start=0, stop=2900, step=1))

----
#### a) How many counties(Län) are represented within the dataset

In [4]:
län_count = df_covid_vaccine.groupby("Län_namn").size() # Here i have grouped the column as series to focus on that information
län_count = län_count.size # .size will then count how long the series is. 
print(f"There are {län_count} different Counties represented in this dataset about Covid 19 Vaccinations.") # Printing out my answer as an f string to include the total count of Läns

There are 21 different Counties represented in this dataset about Covid 19 Vaccinations.


----
#### b) How many municipalities are represented within the dataset

In [5]:
kommun_count = df_covid_vaccine.groupby("Kommun_namn").size() # Using the same method here as with question a)
kommun_count = kommun_count.size
print(f"There are {kommun_count} different municipalities in this dataset about Covid 19 Vaccinations")

There are 290 different  in this dataset about Covid 19 Vaccinations


----
#### c) How big is the Population within this dataset

In [9]:
befolkning = df_covid_vaccine["Befolkning"].sum() # doing a total sum which adds up all values with the population column
befolkning

print(
    f"The total population represented in this dataset is {befolkning} people. This however is not the total population of Sweden as 0-11 year olds are not included."
)


The total population represented in this dataset is 9092790 people. This however is not the total population of Sweden as 0-11 year olds are not included.


----
#### d) Count how many children are under 18 are in Sweden based on the dataset. 

In [12]:
df_grouped_ages = df_covid_vaccine[["Ålder", "Befolkning"]] # Creating a new dataframe for grouping the ages
df_grouped_ages = df_grouped_ages.groupby("Ålder").sum(True).reset_index() # This then groups the ages and takes the total sum for each.I reset the index for order. 

sweden_population = 10246950 # This is the population of Sweden. I took it from a website which i have referenced below.

zero_11 = sweden_population - df_grouped_ages["Befolkning"].sum() # Creating a variable which will provide the missing value of 0-11 ages.
zero_11_row = {"Ålder": "0-11", "Befolkning": zero_11} # Creating a dictionary so the 0-11 row can be added to the dataframe.

df_grouped_ages = (
    df_grouped_ages.append(zero_11_row, ignore_index=True) # addding the new 0-11 row within the datafram using append method
    .sort_values(by="Ålder", ascending=True) # Sorting the dataframe with age order as the new 0-11 was added at the bottom of the rows
    .reset_index(drop=True) # an extra index was added within the process. This will remove that extra index
)

# This code now sums up the groups that are lower then 18 so I can have an amount. This has been assigned to a variable. Thanks Håken Åkeblom for the help with this part 
people_under_18 = df_grouped_ages[
    (df_grouped_ages["Ålder"] == "0-11")
    | (df_grouped_ages["Ålder"] == "12-15")
    | (df_grouped_ages["Ålder"] == "16-17")
]["Befolkning"].sum() # .sum calculates the sum of the chosen cells.

# Printing out my results.
print(
    f"The total amount of people aged under 18 in Sweden is {people_under_18}, The population of Sweden based from https://www.worldometers.info/world-population/sweden-population/ is 10,246,950 as of Tuesday, November 8, 2022 "
)

percentage = people_under_18 / sweden_population * 100 # decided to add a percentage rate to make the results clearer.
print(
    f"This mean that {percentage:.2f} % of People are under the age of 18 in the population of Sweden!"
)


The total amount of people aged under 18 in Sweden is 1899530, The population of Sweden based from https://www.worldometers.info/world-population/sweden-population/ is 10,246,950 as of Tuesday, November 8, 2022 
This mean that 18.54 % of People are under the age of 18 in the population of Sweden!


C:\Users\Sam Glass ITHS\AppData\Local\Temp\ipykernel_7368\1435940522.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_grouped_ages.append(zero_11_row, ignore_index=True) # addding the new 0-11 row within the datafram using append method


In [11]:
# The new dataframe that was created to be used within question d) and e).
df_grouped_ages

,Ålder,Befolkning
0,0-11,1154160
1,12-15,503831
2,16-17,241539
3,18-29,1475950
4,30-39,1467590
5,40-49,1298156
6,50-59,1339798
7,60-69,1121922
8,70-79,1033113
9,80-89,496750


----
#### e) Draw a diagram that shows the ages groups of Swedens Population

In [13]:
# using plotly as you can look at the actual amounts interactively with the cursor
vaccine_by_group = px.bar(
    df_grouped_ages, # using the dataframe that i created with all the age grouos
    x="Ålder",
    y="Befolkning",
    title="Åldersfördelningen av Sveriges befolkning", # Setting a title
    labels={
        "Ålder":"Åldersgrupper" # Making the label more specific
    }
)
vaccine_by_group.show()

----
#### f) Part 1: Draw a bar chart for 1st dose, 2nd dose and 3rd dose proportions for each county 

In [15]:
# First creating a dataframe that has the relative columns i need for the next two questions 
df_grouped = df_covid_vaccine[["Län_namn","Andel minst 1 dos","Andel minst 2 doser","Andel 3 doser","Andel 4 doser"]]
df_grouped = df_grouped.fillna(0) # fixes NaN value to zero so average can be counted correctly
df_grouped.head()

,Län_namn,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,Stockholms län,0.497936,0.431874,0.000000,0.0
1,Stockholms län,0.697423,0.627598,0.000000,0.0
2,Stockholms län,0.730275,0.667812,0.292738,0.0
3,Stockholms län,0.755865,0.714675,0.392526,0.0
4,Stockholms län,0.805068,0.781601,0.535416,0.0


In [20]:
# Now creating a dataframe that groups the counties and counts the average for each cell
df_grouped_counties = df_grouped.groupby("Län_namn").mean("True").reset_index() # This works out the mean average for each dose
df_grouped_counties

,Län_namn,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,Blekinge län,0.856456,0.841993,0.595922,0.248154
1,Dalarnas län,0.855554,0.841344,0.594752,0.264367
2,Gotlands län,0.890340,0.873411,0.622468,0.263385
3,Gävleborgs län,0.851684,0.828887,0.579578,0.253009
4,Hallands län,0.859238,0.844215,0.599350,0.257941
5,Jämtlands län,0.860930,0.843966,0.578451,0.248720
6,Jönköpings län,0.860045,0.846710,0.599758,0.259227
7,Kalmar län,0.851293,0.839464,0.603021,0.258159
8,Kronobergs län,0.824982,0.810197,0.556508,0.245698
9,Norrbottens län,0.870339,0.855377,0.602276,0.254443


In [28]:
proportion_counties = px.bar(
    df_grouped_counties, # Dataframe
    x="Län_namn", # Column
    y=["Andel minst 1 dos","Andel minst 2 doser","Andel 3 doser"], # Using the 3 columns as a list to receive each value
    title="Andel visualisering av Covid 19 Vaccine utfärdats av per Län i Sverige, (Dose 1,2,3) ", # Cresting a title. 
    barmode= "group", # This groups the bars into 3, instead of them being stacked
    labels={
        "Län_namn":"Län namn", # Creating better labels
        "variable": "Andel doser",
        "value":"Summa"}        
    )

proportion_counties.write_html("Visualiseringar/Andel_vaccines_Län.html") # saving graph as html file
proportion_counties.show()

----
#### f) Part 2: Use Västra Göteland county and Stockholms county and draw a diagram representing proportion for dose 1,2,3 and 4 

In [29]:
# Creating a new dataframe to make the plotting process easier for this exercise. It only includes Stockholm and Västra Götelands counties.
df_stock_vgöt = df_grouped_counties.query("Län_namn == ('Västra Götalands län', 'Stockholms län')") # .query allows me to select the required counties
df_stock_vgöt

,Län_namn,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
11,Stockholms län,0.835868,0.805936,0.568321,0.242257
18,Västra Götalands län,0.859421,0.839688,0.577908,0.253415


In [32]:
proportion_Stock_Väst = px.bar(
    df_stock_vgöt,
    x="Län_namn",
    y=["Andel minst 1 dos","Andel minst 2 doser","Andel 3 doser","Andel 4 doser"],
    title="Andel visualisering av Covid 19 Vaccine utfärdats av per Västra Götaland och Stockholmns Län, (Dose 1,2,3,4)",
    barmode= "group",
    labels={
        "Län_namn":"Län Namn",
        "value":"Antal"       
    }
)
proportion_counties.write_html("Visualiseringar/Andel_vaccines_Stock_VGöt.html") # saving graph as html file
proportion_Stock_Väst.show()

References and research: 
- https://www.statology.org/
- https://plotly.com/
- https://www.geeksforgeeks.org/
- https://stackoverflow.com/

These were the main websites that I have used within the projects to help with coding ideas and methods.